#### **1. Define System**

In [1]:
from jqme import System

#### **2. Define Environment**

In [2]:
class Environment:
    pass

#### **3. Define evolution**

In [3]:
import numpy as np
from scipy.linalg import toeplitz

epsilon = [1, 2]
J = toeplitz([0, 1])


sys = System(energies=epsilon,
             couplings=J,
             )

In [4]:
print(sys.energies)
print(sys.couplings)
print(sys.dipole_moments)

[1 2]
[[0 1]
 [1 0]]
[1. 1.]


In [5]:
sys.get_e_Hamiltonian()

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=CSR, isherm=True
Qobj data =
[[-1.5  0.   0.   0. ]
 [ 0.  -0.5  1.   0. ]
 [ 0.   1.   0.5  0. ]
 [ 0.   0.   0.   1.5]]

In [6]:
sys.set_state(state_type='delocalized excitation',
              state=[1/np.sqrt(2), 1/np.sqrt(2)])

sys.get_e_state()

Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.        ]
 [0.70710678]
 [0.70710678]
 [0.        ]]

In [7]:
sys.to_dict()

{'class': 'System',
 'validity': True,
 'quantities': {'energies': array([1, 2]),
  'system_size': 2,
  'couplings': array([[0, 1],
         [1, 0]]),
  'dipoles': array([1., 1.]),
  'state_type': 'delocalized excitation',
  'state': [(0.7071067811865475+0j), (0.7071067811865475+0j)]}}